In [ ]:
import time
from datetime import datetime, timedelta

def run_strategy():
    # Execute the strategy immediately on startup
    print("Running strategy at", datetime.now())
    # Add your trading strategy logic here
    # Make a csv that contains the first set of data for the optimization

    while True:
        # Get current time and calculate next hour
        now = datetime.now()
        next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

        # Wait until the next hour
        sleep_duration = (next_hour - now).total_seconds()
        print(f"Sleeping for {sleep_duration} seconds...")
        time.sleep(sleep_duration)

        # Execute your strategy
        print("Running strategy at", datetime.now())
        # Add your trading strategy logic here

run_strategy() # call the function

In [ ]:
import ccxt
import pandas as pd
import os
import time
from unsync import unsync

# Initialize Kraken Pro Exchange
exchange = ccxt.kraken({
    'apiKey': 'your_api_key',
    'secret': 'your_api_secret'
})

# Fetch the latest OHLCV data point (asynchronous)
@unsync
def fetch_latest_data(symbol, timeframe):
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=1)
        latest_data = ohlcv[-1]
        return {
            'timestamp': pd.to_datetime(latest_data[0], unit='ms'),
            'open': latest_data[1],
            'high': latest_data[2],
            'low': latest_data[3],
            'close': latest_data[4],
            'volume': latest_data[5]
        }
    except Exception as e:
        print(f"Error fetching latest data: {e}")
        return None

# Append new data to CSV and maintain max length (asynchronous)
@unsync
def append_to_csv_with_limit(data, filename, max_rows=3000):
    file_exists = os.path.isfile(filename)
    df = pd.DataFrame([data])
    
    if file_exists:
        existing_df = pd.read_csv(filename)
        combined_df = pd.concat([existing_df, df], ignore_index=True)
        if len(combined_df) > max_rows:
            combined_df = combined_df.iloc[-max_rows:]  # Keep only the last max_rows rows
        combined_df.to_csv(filename, index=False)
    else:
        df.to_csv(filename, mode='w', header=True, index=False)

# Load the dataset from the CSV file (synchronous, needed for signal calculations)
def load_data_from_csv(filename):
    if os.path.isfile(filename):
        return pd.read_csv(filename)
    else:
        return pd.DataFrame()

# Calculate moving averages on the dataset
def calculate_moving_averages(df, fast_period, slow_period):
    df['fast_ma'] = df['close'].rolling(window=fast_period).mean()
    df['slow_ma'] = df['close'].rolling(window=slow_period).mean()
    return df

# Check crossover signals
def check_crossover(df):
    if len(df) < 2:
        return None
    if df['fast_ma'].iloc[-2] < df['slow_ma'].iloc[-2] and df['fast_ma'].iloc[-1] > df['slow_ma'].iloc[-1]:
        return 'buy'
    elif df['fast_ma'].iloc[-2] > df['slow_ma'].iloc[-2] and df['fast_ma'].iloc[-1] < df['slow_ma'].iloc[-1]:
        return 'sell'
    else:
        return None


def run_strategy_logic(symbol, timeframe, fast_period, slow_period, filename):
    """Executes the core strategy logic (fetching, appending, calculating, checking)."""
    fetch_task = fetch_latest_data(symbol, timeframe)
    latest_data = fetch_task.result()

    if latest_data:
        append_task = append_to_csv_with_limit(latest_data, filename)  # Use default max_rows
        append_task.result()

        dataset = load_data_from_csv(filename)
        dataset = calculate_moving_averages(dataset, fast_period, slow_period)
        signal = check_crossover(dataset)

        if signal == 'buy':
            print(f"Buy signal detected at {latest_data['timestamp']}")
            # Place buy order logic here
        elif signal == 'sell':
            print(f"Sell signal detected at {latest_data['timestamp']}")
            # Place sell order logic here
        else:
            print(f"No signal at {latest_data['timestamp']}")

def run_hourly_strategy(symbol, timeframe, fast_period, slow_period, filename):
    """Runs the strategy hourly."""

    # Run the strategy logic immediately on startup
    print("Running strategy on startup at:", datetime.now())
    run_strategy_logic(symbol, timeframe, fast_period, slow_period, filename)

    while True:
        now = datetime.now()
        next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
        sleep_duration = (next_hour - now).total_seconds()
        print(f"Sleeping for {sleep_duration} seconds until the next hour...")
        time.sleep(sleep_duration)

        print("Running strategy at:", datetime.now())
        run_strategy_logic(symbol, timeframe, fast_period, slow_period, filename)

# Run the strategy

symbol = 'BTC/USD'  # Trading pair
timeframe = '1m'  # Timeframe
fast_period = 5  # Fast moving average period
slow_period = 20  # Slow moving average period
csv_filename = 'market_data.csv'  # File to store historical data
max_data_points = 3000  # Maximum number of rows in the CSV

run_hourly_strategy(symbol, timeframe, fast_period, slow_period, csv_filename)

### Process:
1. Ensure that we are trading every hour
2. Get live data (3 Data points to account for the data preparation)
3. Append it to csv, ensuring the limit of data points in the csv file (the limit == the train_size for the optimization process)
4. Load the data from the csv file
5. optimize if time to optimize (use optimize_counter)
6. rebalance if time to rebalance (use rebalance_counter)
7. Run the strategy on the dataset
8. On the last time index (the last candle), get the universe (level 2 index) and actual allocation for each coin\
=> Make sure that we have applied self.live = True when initiating the strategy (to not shift the position values)\
(a) If coin not in universe -> Put their allocation\
(b) If actual allocation != shifted allocation, change the allocation of the coin\
--> Changing the allocation of a coin, is by determining the amount in currency = (amount_USD / close_price) to sell or to buy, then placing that order

In [1]:
import ccxt
import pandas as pd
import os
import time
from unsync import unsync
from datetime import datetime, timedelta

In [2]:
#Connect to exchange
api_key = 'yqPWrtVuElaIExKmIp/E/upTOz/to1x7tC3JoFUxoSTKWCOorT6ifF/B'
api_secret = 'L8h5vYoAu/jpQiBROA9yKN41FGwZAGGVF3nfrC5f5EiaoF7VksruPVdD7x1VOwnyyNCMdrGnT8lP4xHTiBrYMQ=='

exchange = ccxt.kraken({
    'apiKey': api_key,
    'secret': api_secret
})

In [3]:
#Converting symbols to a format that the exchange understands
symbols = ['BTCUSD', 'ETHUSD', 'LTCUSD']
formatted_symbol = [symbol.replace("USD", "/USD") for symbol in symbols]
formatted_symbol

['BTC/USD', 'ETH/USD', 'LTC/USD']

In [115]:
#Account Portfolio operations
balance = exchange.fetch_balance()
balance

{'info': {'error': [],
  'result': {'XXBT': {'balance': '0.0000000000', 'hold_trade': '0.0000000000'},
   'ZCAD': {'balance': '0.0000', 'hold_trade': '0.0000'},
   'ZUSD': {'balance': '33.6557', 'hold_trade': '0.0000'}}},
 'timestamp': None,
 'datetime': None,
 'BTC': {'free': 0.0, 'used': 0.0, 'total': 0.0},
 'CAD': {'free': 0.0, 'used': 0.0, 'total': 0.0},
 'USD': {'free': 33.6557, 'used': 0.0, 'total': 33.6557},
 'free': {'BTC': 0.0, 'CAD': 0.0, 'USD': 33.6557},
 'used': {'BTC': 0.0, 'CAD': 0.0, 'USD': 0.0},
 'total': {'BTC': 0.0, 'CAD': 0.0, 'USD': 33.6557}}

In [5]:
balance['total']['USD']

33.6557

In [7]:
ohlcv = exchange.fetch_ohlcv('BTC/USD', '1h', limit=10)
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])

In [8]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)

In [9]:
df[:-1]

,open,high,low,close,volume
timestamp,,,,,
2025-01-11 13:00:00,94551.4,94551.9,94354.6,94356.8,10.062514
2025-01-11 14:00:00,94356.8,94549.1,94284.0,94548.3,5.791742
2025-01-11 15:00:00,94548.3,94580.0,94334.7,94359.3,19.460417
2025-01-11 16:00:00,94359.3,94665.5,94330.1,94391.2,7.116978
2025-01-11 17:00:00,94401.8,94401.8,93919.0,94104.2,19.314240
2025-01-11 18:00:00,94104.2,94257.0,94075.0,94244.1,32.384815
2025-01-11 19:00:00,94244.1,94281.3,94007.6,94187.5,6.698506
2025-01-11 20:00:00,94187.5,94597.4,94160.4,94567.7,13.310514
2025-01-11 21:00:00,94567.7,94986.6,94530.1,94921.6,65.167654


In [10]:
@unsync
def fetch_latest_data(symbol, timeframe):
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=3)
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        latest_data = df[:-1]
        return latest_data
    except Exception as e:
        print(f"Error fetching latest data: {e}")
        return None
    
# Fetch the latest OHLCV data point (asynchronous)
try:
    latest = fetch_latest_data('BTC/USD', '1h')
    latest.result()
    print('latest data:', latest.result())
except Exception as e:
    print(f"Error fetching latest data: {e}")

latest data:                         open     high      low    close     volume
timestamp                                                         
2025-01-11 20:00:00  94187.5  94597.4  94160.4  94567.7  13.310514
2025-01-11 21:00:00  94567.7  94986.6  94530.1  94921.6  65.167654


In [11]:
latest.result()

,open,high,low,close,volume
timestamp,,,,,
2025-01-11 20:00:00,94187.5,94597.4,94160.4,94567.7,13.310514
2025-01-11 21:00:00,94567.7,94986.6,94530.1,94921.6,65.167654


In [ ]:
@unsync
def fetch_latest_data(symbols, timeframe, limit = 2):
    """Fetch latest OHLCV data for multiple symbols and stack them into a single DataFrame."""
    data_frames = []
    for symbol in symbols:
        try:
            # Fetch OHLCV data
            ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
            df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            
            # Convert timestamp to datetime
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)
            
            # Add a symbol column
            df['coin'] = symbol
            data_frames.append(df)
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
    
    # Concatenate all DataFrames and set multi-level index
    if data_frames:
        stacked_df = pd.concat(data_frames)
        stacked_df.set_index('coin', append=True, inplace=True)
        # Remove duplicates from the index
        stacked_df = stacked_df[~stacked_df.index.duplicated()]
        df = data_instance.prepare_data(stacked_df.unstack())
        return df
    else:
        return pd.DataFrame()  # Return an empty DataFrame if no data

# Example usage
timeframe = '1h'
latest = fetch_latest_data(symbols, timeframe)
latest.result()


In [12]:
latest.result()['close'].iloc[-1]

94921.6

In [73]:
#Placing a limit order
symbol = "BTC/USD"  # Trading pair
order_type = "limit"  # Order type: limit or market
side = "buy"  # Order side: buy or sell
amount = balance['total']['USD'] / latest.result()['close'].iloc[-1]  # Amount in BTC
price = latest.result()['close'].iloc[-1] # Limit price in USD

try:
    order = exchange.create_order(symbol, order_type, side, amount, price)
    print("Order placed successfully:", order)
except Exception as e:
    print(f"Error placing order: {e}")

Order placed successfully: {'id': 'O23OWI-A2ULZ-BWA3VH', 'clientOrderId': None, 'info': {'txid': ['O23OWI-A2ULZ-BWA3VH'], 'descr': {'order': 'buy 0.00036295 XBTUSD @ limit 94187.5'}}, 'timestamp': None, 'datetime': None, 'lastTradeTimestamp': None, 'status': None, 'symbol': 'BTC/USD', 'type': 'limit', 'timeInForce': None, 'postOnly': False, 'side': 'buy', 'price': 94187.5, 'stopPrice': None, 'triggerPrice': None, 'takeProfitPrice': None, 'stopLossPrice': None, 'cost': None, 'amount': 0.00036295, 'filled': None, 'average': None, 'remaining': None, 'reduceOnly': None, 'fee': None, 'trades': [], 'fees': [], 'lastUpdateTimestamp': None}


In [13]:
#To fetch open orders
try:
    # Fetch open orders
    open_orders = exchange.fetch_open_orders()
    for order in open_orders:
        print(f"Order ID: {order['id']}, Symbol: {order['symbol']}, Amount: {order['amount']}")
except Exception as e:
    print(f"Error fetching open orders: {e}")

In [77]:
#To cancel an order
try:
    order_id = "O23OWI-A2ULZ-BWA3VH"  # Replace with the actual order ID
    cancel_response = exchange.cancel_order(order_id)
    print(f"Order canceled successfully: {cancel_response}")
except Exception as e:
    print(f"Error canceling order: {e}")


Order canceled successfully: {'info': {'error': [], 'result': {'count': '1'}}, 'fees': [], 'id': None, 'clientOrderId': None, 'timestamp': None, 'datetime': None, 'symbol': None, 'type': None, 'side': None, 'lastTradeTimestamp': None, 'lastUpdateTimestamp': None, 'price': None, 'amount': None, 'cost': None, 'average': None, 'filled': None, 'remaining': None, 'timeInForce': None, 'postOnly': None, 'trades': [], 'reduceOnly': None, 'stopPrice': None, 'triggerPrice': None, 'takeProfitPrice': None, 'stopLossPrice': None, 'status': None, 'fee': None}


In [113]:
symbol = "BTC/USD"  # Replace with your desired symbol
try:
    # Specify the ticker symbol
    ticker = exchange.fetch_ticker(symbol)
    
    # Get the current price
    current_price = ticker['last']  # 'last' is the latest trade price
    print(f"Current price of {symbol}: {current_price}")
except Exception as e:
    print(f"Error fetching ticker data: {e}")

Current price of BTC/USD: 92222.1


In [116]:
# Placing a buy order and a sell order
order_type = "market"  # Market order
symbol = "BTC/USD"  # Trading pair
side = "buy"  # Order side: buy or sell
amount = balance['total']['USD'] / current_price  # Amount in BTC

try:
    order = exchange.create_order(symbol, order_type, side, amount)
    print("Market order placed successfully:", order)
except Exception as e:
    print(f"Error placing market order: {e}")

Market order placed successfully: {'id': 'ORBCS3-XUKDQ-KIXTK4', 'clientOrderId': None, 'info': {'txid': ['ORBCS3-XUKDQ-KIXTK4'], 'descr': {'order': 'buy 0.00036494 XBTUSD @ market'}}, 'timestamp': None, 'datetime': None, 'lastTradeTimestamp': None, 'status': None, 'symbol': 'BTC/USD', 'type': 'market', 'timeInForce': 'IOC', 'postOnly': False, 'side': 'buy', 'price': None, 'stopPrice': None, 'triggerPrice': None, 'takeProfitPrice': None, 'stopLossPrice': None, 'cost': None, 'amount': 0.00036494, 'filled': None, 'average': None, 'remaining': None, 'reduceOnly': None, 'fee': None, 'trades': [], 'fees': [], 'lastUpdateTimestamp': None}


In [15]:
symbols_to_trade = [symbol.replace("/USD", "") for symbol in formatted_symbol]

In [16]:
symbols_to_trade

['BTC', 'ETH', 'LTC']

In [17]:
balance = exchange.fetch_balance()
balance['total'][symbols_to_trade[0]]

0.0

In [18]:
balance

{'info': {'error': [],
  'result': {'XXBT': {'balance': '0.0000000000', 'hold_trade': '0.0000000000'},
   'ZCAD': {'balance': '0.0000', 'hold_trade': '0.0000'},
   'ZUSD': {'balance': '33.6557', 'hold_trade': '0.0000'}}},
 'timestamp': None,
 'datetime': None,
 'BTC': {'free': 0.0, 'used': 0.0, 'total': 0.0},
 'CAD': {'free': 0.0, 'used': 0.0, 'total': 0.0},
 'USD': {'free': 33.6557, 'used': 0.0, 'total': 33.6557},
 'free': {'BTC': 0.0, 'CAD': 0.0, 'USD': 33.6557},
 'used': {'BTC': 0.0, 'CAD': 0.0, 'USD': 0.0},
 'total': {'BTC': 0.0, 'CAD': 0.0, 'USD': 33.6557}}

In [19]:
try:
    # Step 1: Check your balance
    balance = exchange.fetch_balance()
    coin = "BTC"  # Replace with the coin you want to sell
    amount = balance[coin]["free"]  # Amount available to sell
    print(f"Available {coin}: {amount}")

    if amount > 0:
        # Step 2: Place a market sell order
        symbol = formatted_symbol[0]  # Replace USD with your desired quote currency
        order = exchange.create_market_sell_order(symbol, amount)
        print(f"Sell order placed: {order}")
    else:
        print(f"No {coin} available to sell.")
except Exception as e:
    print(f"Error: {e}")

Available BTC: 0.0
No BTC available to sell.


In [ ]:
def run_hourly_strategy(symbol, timeframe, fast_period, slow_period, filename):
    """Runs the strategy hourly."""

    # Run the strategy logic immediately on startup
    print("Running strategy on startup at:", datetime.now())
    # run_strategy_logic(symbol, timeframe, fast_period, slow_period, filename)

    while True:
        now = datetime.now()
        next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
        print(f"Next hour: {next_hour}")
        sleep_duration = (next_hour - now).total_seconds()
        print(f"Sleeping for {sleep_duration} seconds until the next hour...")
        time.sleep(sleep_duration)

        print("Running strategy at:", datetime.now())
        # run_strategy_logic(symbol, timeframe, fast_period, slow_period, filename)
        
symbol = 'BTC/USD'  # Trading pair
timeframe = '1m'  # Timeframe
fast_period = 5  # Fast moving average period
slow_period = 20  # Slow moving average period
csv_filename = 'market_data.csv'  # File to store historical data
max_data_points = 3000  # Maximum number of rows in the CSV

run_hourly_strategy(symbol, timeframe, fast_period, slow_period, csv_filename)

In [95]:
#To liquidate everything:
try:
    # Step 1: Get your balances
    balance = exchange.fetch_balance()
    
    # Step 2: Loop through all assets in your balance and sell them
    for coin, coin_balance in balance['free'].items():
        if coin_balance > 0:  # Only sell if you have a non-zero balance
            print(f"Selling {coin_balance} {coin}...")
            
            # Determine the symbol for the sell order (e.g., BTC/USD, ETH/USDT)
            symbol = f"{coin}/USD"  # Replace USD with your preferred quote currency
            order = exchange.create_market_sell_order(symbol, coin_balance)
            print(f"Sell order placed: {order}")
        else:
            print(f"No {coin} to sell.")
    
    print("All possible assets have been liquidated.")
    
except Exception as e:
    print(f"Error: {e}")

No BTC to sell.
No CAD to sell.
Selling 33.6557 USD...
Error: kraken does not have market symbol USD/USD


In [20]:
def liquidate(symbols, exchange):
    try:
        # Step 1: Get your balances
        balance = exchange.fetch_balance()

        # Step 2: Loop through all assets in your balance and sell them
        for coin, coin_balance in balance['free'].items():
            if coin in symbols:
                if coin_balance > 0:  # Only sell if you have a non-zero balance
                    print(f"Selling {coin_balance} {coin}...")

                    # Determine the symbol for the sell order (e.g., BTC/USD, ETH/USDT)
                    symbol = f"{coin}/USD"  # Replace USD with your preferred quote currency
                    order = exchange.create_market_sell_order(symbol, coin_balance)
                    print(f"Sell order placed: {order}")
                else:
                    print(f"No {coin} to sell.")

        print("All possible assets have been liquidated.")

    except Exception as e:
        print(f"Error: {e}")

---

### Pseudocode:

```(Python)
#in the hourly function loop
self.counter_opt = 0
self.counter_reb = 0
self.strats_map = {...}
self.selected_strats = {...}
self.weights = []
self.best_params = {}

run_strategy():
    opt_interval = strat.train_size
    reb_interval = 236 #nbr of hours in each 2 weeks


    latest = fetch_last_data()\
    latest.result()

    if latest:
        append = append_to_csv(latest)
        append.result() #Because we are going to be using the unsync the library

        data = load_data_from_csv()

        if self.counter_opt % opt_interval == 0 and len(data) >= opt_interval:
            self.best_params = [strat.optimize() for strat in self.strats_map]

        
        self.opt_counter += 1
        self.reb_counter += 1

        df_strats = [
        strat(self.best_params[i])
        for i in range(len(self.best_params))  # Iterate over indices
        for strat in self.selected_strats.values()  # Iterate over strategies
        ]

        if self.counter_reb % reb_interval == 0 and len(data) >= reb_interval:
            log_rets = [strat.trading_strategy(data, self.best_params).results.strategy for strat in self.selected_strats.values()]
            self.selected_strats = portfolio_management(strat_map) -> {}
            weights = portfolio_optimization(log_rets, ...).optimize(data) -> []
            portfolio_value = exchange.... #the current portfolio value of the account
            max_allocation_per_strat_list = weights * list -> []
            self.max_allocation_per_strat_map = dict(zip(self.selected_strats.keys(), max_allocation_per_strat_list))

        #actual_allocation in coin currency
        for i in range(df_strats):
            df['actual_allocation_coin_currency'] = df['actual_allocation']  / df['close']

        current_universes= [df_strats[i].index.get_level_values(1).unique() for i in range(df_strats)]

        # Flatten into 1D list
        current_universe = []
        for sublist in two_d_list:
            for item in sublist:
                current_universe.append(item) if item not in current_universe
            
        current_coins = exchange.... #Current coins in the account portfolio

        for coin in current_coins
            if coin not in current_universe:
                #liquidate those coins

        coins_allocations = sum(df_strats['actual_allocation_coin_currency']).iloc[-1]

        for coin in current_universe:
            current_position = exchange.... -> float()
            actual_allocation = coins_allocation.unstack().coin -> float()

            to_add = current_position - actual_allocation

            if to_add > 0:
                #place a buy order of that quantity for that coin
            if to_add < 0:
                #place a sell order of that quantity for that coin

In [1]:
import ccxt
import pandas as pd
import numpy as np
import os
import time
from unsync import unsync
import datetime as dt
import sys
from concurrent.futures import ThreadPoolExecutor
import warnings
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)

In [2]:
# Ensure the directories are in the system path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'Data_Management'))) 
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'Portfolio_Optimization'))) 
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'Strategies', 'Trend_Following')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'Strategies', 'Mean_Reversion')))

# Import the modules
from data import Data, get_halal_symbols
from fetch_symbols import get_symbols
from sprtrnd_breakout import Sprtrnd_Breakout
from last_days_low import Last_Days_Low
from portfolio_management import Portfolio_Management
from portfolio_optimization import Portfolio_Optimization
from portfolio_risk_management import Portfolio_RM


### At boot

In [3]:
#Connect to exchange
api_key = 'yqPWrtVuElaIExKmIp/E/upTOz/to1x7tC3JoFUxoSTKWCOorT6ifF/B'
api_secret = 'L8h5vYoAu/jpQiBROA9yKN41FGwZAGGVF3nfrC5f5EiaoF7VksruPVdD7x1VOwnyyNCMdrGnT8lP4xHTiBrYMQ=='

exchange = ccxt.kraken({
    'apiKey': api_key,
    'secret': api_secret
})

In [4]:
all_symbols = get_symbols()

In [5]:
train_size = 2201
test_size = 2201

In [79]:
start_time = (dt.datetime.now() - dt.timedelta(hours=train_size + test_size + 200)).date()
end_time = dt.datetime.now().date()
timeframes = ['1w', '1d', '4h', '1h', '30m','15m', '5m', '1m']
index = 3 #It is better to choose the highest frequency for the backtest to be able to downsample
interval = timeframes[index]
symbols = ['BTCUSD', 'ETHUSD', 'BNBUSD', 'ADAUSD', 'XRPUSD']
data_instance = Data(all_symbols, interval, start_time, end_time, exchange = 'kraken')

c:\Users\yassi\OneDrive\Documents\GitHub\Portfolio_1\Technical_Portfolio\Data_Management\data.py:104: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  _df['returns', coin] = _df['close', coin].pct_change()
c:\Users\yassi\OneDrive\Documents\GitHub\Portfolio_1\Technical_Portfolio\Data_Management\data.py:104: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  _df['returns', coin] = _df['close', coin].pct_change()
c:\Users\yassi\OneDrive\Documents\GitHub\Portfolio_1\Technical_Portfolio\Data_Management\data.py:104: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be re

In [80]:
data = data_instance.df

In [81]:
last_date_data = data.index.get_level_values(0).unique()[-1].tz_localize('UTC')

In [82]:
formatted_symbols = format_symbols(symbols)

In [89]:
if dt.datetime.now(dt.UTC).replace(minute=0, second=0, microsecond=0) != last_date_data:
    time_difference = dt.datetime.now(dt.UTC).replace(minute=0, second=0, microsecond=0) - last_date_data
    hours_difference = time_difference.total_seconds() / 3600 # Get the number of hours
    missing_data = fetch_latest_data(formatted_symbols, interval, limit = int(hours_difference) + 2).result()
    complete_data = pd.concat([data, missing_data])

In [92]:
complete_data.index = complete_data.index.set_levels(pd.to_datetime(complete_data.index.levels[0]), level=0)

In [94]:
complete_data.to_csv('market_data.csv')

### At each time we loop

In [125]:
#Helper function
def get_last_row(data):
    """Get the last date in the dataset."""
    last_date = data.index.get_level_values("date").max()
    last_date_data = data.loc[last_date]
    return last_date_data

In [122]:
def get_portfolio_value(exchange):
    try:
        # Fetch account balances
        balances = exchange.fetch_balance()

        # Fetch tickers to get the latest prices
        tickers = exchange.fetch_tickers()

        # Calculate portfolio value in USD (or another base currency)
        portfolio_value = 0.0

        for currency, balance in balances['total'].items():
            if balance > 0:
                # Use the USD pair or the most liquid market
                pair = f"{currency}/USD"
                if pair in tickers:
                    price = tickers[pair]['last']
                    portfolio_value += balance * price
                else:
                    # Handle currencies without USD pairs (e.g., trade to BTC, then USD)
                    btc_pair = f"{currency}/BTC"
                    if btc_pair in tickers:
                        btc_price = tickers[btc_pair]['last']
                        usd_price = tickers["BTC/USD"]['last']
                        portfolio_value += balance * btc_price * usd_price

        return portfolio_value

    except ccxt.BaseError as e:
        print(f"An error occurred: {str(e)}")
        return None

In [123]:
portfolio_value = get_portfolio_value(exchange)

In [124]:
portfolio_value

33.38000406

In [20]:
def format_symbols(symbols):
    """Converts the symbols to a format that the exchange understands."""
    if symbols[0].endswith('T'):
        symbols = [s[:-1] for s in symbols]
    formatted_symbols = [symbol.replace("USD", "/USD") for symbol in symbols]
    return formatted_symbols

def filter_halal_df(data):
    # Drop multiple coins
    halal_symbols = ['BTC/USD', 'ETH/USD', 'LTC/USD']
    data_filtered = data[data.index.get_level_values("coin").isin(halal_symbols)]
    return data_filtered

In [ ]:
halal_symbols = get_halal_symbols()

In [ ]:
formatted_halal_symbols = format_symbols(halal_symbols)

In [31]:
@unsync
def fetch_latest_data(symbols, timeframe, limit=2):
    """Fetch latest OHLCV data for multiple symbols and stack them into a single DataFrame."""
    
    def fetch_symbol_data(symbol):
        """Fetch data for a single symbol and return a DataFrame."""
        try:
            ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
            df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)
            df['coin'] = symbol
            return df
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            return pd.DataFrame()  # Return an empty DataFrame if fetching fails

    # Use ThreadPoolExecutor for parallel requests
    with ThreadPoolExecutor(max_workers=16) as executor:  # Adjust workers based on CPU
        results = list(executor.map(fetch_symbol_data, symbols))

    # Concatenate all DataFrames and set multi-level index
    data_frames = [df for df in results if not df.empty]
    if data_frames:
        stacked_df = pd.concat(data_frames)
        stacked_df.set_index('coin', append=True, inplace=True)
        stacked_df = stacked_df[~stacked_df.index.duplicated()]  # Remove duplicates
        df = data_instance.prepare_data(stacked_df.unstack())
        return df
    else:
        return pd.DataFrame()  # Return an empty DataFrame if no data

# Example usage
timeframe = '1h'
latest = fetch_latest_data(formatted_symbols, timeframe)
sample_data = latest.result()

In [32]:
sample_data

close  creturns          high  log_return  \
date                coin                                                        
2025-01-13 19:00:00 ADA/USD      0.912936  1.005705      0.917609    0.005689   
                    BTC/USD  92090.700000  1.000700  92442.900000    0.000700   
                    ETH/USD   3022.840000  1.005191   3034.570000    0.005177   
                    XRP/USD      2.464430  1.006391      2.480670    0.006371   

                                      low          open         price  \
date                coin                                                
2025-01-13 19:00:00 ADA/USD      0.903511      0.906877      0.912936   
                    BTC/USD  91605.400000  92026.300000  92090.700000   
                    ETH/USD   2992.410000   3007.230000   3022.840000   
                    XRP/USD      2.438000      2.448960      2.464430   

                              returns        volume  volume_in_dollars  
date                coin                                                
2025-01-13 19:00:00 ADA/USD  0.005705  7.747882e+05       7.073320e+05  
                    BTC/USD  0.000700  4.590086e+01       4.227043e+06  
                    ETH/USD  0.005191  2.572654e+02       7.776721e+05  
                    XRP/USD  0.006391  2.154674e+06       5.310042e+06

In [102]:
# Append new data to CSV and maintain max length (asynchronous)
@unsync
def append_to_csv_with_limit(data, filename, max_rows=2202):
    file_exists = os.path.isfile(filename)
    df = pd.DataFrame(data)
    
    if file_exists:
        existing_df = pd.read_csv(filename, index_col=['date', 'coin'], parse_dates=['date'])
        combined_df = pd.concat([existing_df, df])
        if len(combined_df) > max_rows:
            combined_df = combined_df.iloc[-max_rows:]  # Keep only the last max_rows rows
        combined_df.to_csv(filename)
    else:
        print('File does not exist')
        df.to_csv(filename, mode='w', header=True)


In [4]:
#Getting the data from csv
def load_data_from_csv(filename, train_size = 2000, test_size = 2000):
    if os.path.isfile(filename):
        data = pd.read_csv(filename, index_col=['date', 'coin'], parse_dates=['date'])
        if len(data) >= train_size + test_size:
            return data
    else:
        return pd.DataFrame()

In [13]:
market_data = load_data_from_csv('market_data.csv')

### Portfolio Management

Should be ran every time we want to optimize

In [104]:
mr_strat_1 = Last_Days_Low(complete_data, objective = 'multiple')
tf_strat_1 = Sprtrnd_Breakout(complete_data, objective = 'multiple')

#Create a dummy results that represents holding cash where the value of the portfolio is constant
cash_df = pd.DataFrame(data = {'strategy': np.zeros(data.shape[0]), 'portfolio_value': np.ones(data.shape[0])}, index = data.index)

strategy_map = {'cash_strat': cash_df,
                'mr_strat_1': mr_strat_1,
                'tf_strat_1': tf_strat_1}

In [105]:
#Run the WFO for each strategy (but the cash strategy)
for key, value in strategy_map.items():
    if key != 'cash_strat':
        value.test()

TypeError: '>' not supported between instances of 'float' and 'NoneType'

In [ ]:
#Make a new dictionary that contains the results strategy returns of the WFO
results_strategy_returns = {}
for key, value in strategy_map.items():
    if key != 'cash_strat':
        results_strategy_returns[key] = value.results.strategy
    elif key == 'cash_strat':
        results_strategy_returns[key] = value.strategy

In [ ]:
portfolio_management = Portfolio_Management(results_strategy_returns)

In [ ]:
keys_for_selected_strategy = portfolio_management.filter_by_correlation().columns

In [ ]:
selected_strategy = {key: value for key, value in strategy_map.items() if key in keys_for_selected_strategy}

### Strategy Optimization

In [ ]:
counter_opt = 0
counter_reb = 0 

In [ ]:
#Run the optimization to get the strategy parameters
